In [ ]:
import pandas as pd

psychosocialworkload = pd.read_csv(r'C:\Users\mth2\OneDrive - Gemeente Breda\Bureaublad\Github\uwv\data\83157NED_TypedDataSet_04102024_150331.csv', delimiter=';')
psychosocialworkload = psychosocialworkload.drop(['ID', 'Marges'], axis=1)
psychosocialworkload['BedrijfskenmerkenSBI2008'] = psychosocialworkload['BedrijfskenmerkenSBI2008'].str.strip('T') #om te zorgen dat alle rijen meegenomen worden, anders waren het alleen de eerste 10 (de daadwerkelijke objects)
psychosocialworkload['BedrijfskenmerkenSBI2008'] = psychosocialworkload['BedrijfskenmerkenSBI2008'].astype(float)

In [ ]:
# Extract the year from the 'Perioden' column
psychosocialworkload['Jaar'] = psychosocialworkload['Perioden'].str[:4]  # Extract first 4 characters as year

In [ ]:
psychosocialworkload

In [ ]:
def convert_yearly_to_quarterly(psychosocialworkload):
    """
    Converts yearly data to quarterly data by duplicating each row four times 
    and assigning quarters KW01 to KW04 to the 'Kwartaal' column for each duplicated row.

    Parameters:
        DataFrame: The DataFrame containing yearly data.

    Returns:
        DataFrame: The DataFrame with yearly data converted to quarterly data.
    """
    # Step 1: Create a list of quarters (1, 2, 3, 4) for each year
    quarters = ['KW01', 'KW02', 'KW03', 'KW04']

    # Step 2: Repeat each row 4 times and create a new column 'Kwartaal' with the corresponding quarter
    df_quarterly = psychosocialworkload.loc[psychosocialworkload.index.repeat(4)].reset_index(drop=True)
    df_quarterly['Kwartaal'] = quarters * len(psychosocialworkload)

    return df_quarterly

psychosocialworkload_quarter = convert_yearly_to_quarterly(psychosocialworkload)

psychosocialworkload_quarter

In [ ]:
psychosocialworkload_quarter['Perioden_nieuw'] = psychosocialworkload_quarter['Jaar'].astype(str) + psychosocialworkload_quarter['Kwartaal']

In [ ]:
psychosocialworkload_quarter

In [ ]:
psychosocialworkload_quarter = psychosocialworkload_quarter.drop(columns=['Perioden', 'Jaar', 'Kwartaal'])

In [ ]:
psychosocialworkload_quarter = psychosocialworkload_quarter.rename(columns={'Perioden_nieuw': 'Perioden'})

In [ ]:
psychosocialworkload_quarter

In [ ]:
sickleave = pd.read_csv(r'C:\Users\mth2\OneDrive - Gemeente Breda\Bureaublad\Github\uwv\data\external\cbs\80072ned\80072ned_TypedDataSet.csv', delimiter=',')
sickleave = sickleave.drop('ID', axis=1)
sickleave['BedrijfskenmerkenSBI2008'] = sickleave['BedrijfskenmerkenSBI2008'].str.strip('T') #om te zorgen dat alle rijen meegenomen worden, anders waren het alleen de eerste 10 (de daadwerkelijke objects)
sickleave['BedrijfskenmerkenSBI2008'] = sickleave['BedrijfskenmerkenSBI2008'].str.strip('WP') #om te zorgen dat alle rijen meegenomen worden, anders waren het alleen de eerste 10 (de daadwerkelijke objects)
sickleave['BedrijfskenmerkenSBI2008'] = sickleave['BedrijfskenmerkenSBI2008'].astype(float)
sickleave

In [ ]:
merged_df = pd.merge(psychosocialworkload_quarter, sickleave, on=['BedrijfskenmerkenSBI2008', 'Perioden'])

merged_df

In [ ]:
merged_df.info()

In [ ]:
columns_to_compare = ['Ziekteverzuimpercentage_1', 'ErgSnelWerken_1', 'HeelVeelWerkDoen_2', 'ExtraHardWerken_3', 'ZelfBeslissen_4', 'VolgordeWerkzaamhedenBepalen_5', 'WerktempoBepalen_6', 'OplossingenBedenken_7', 'VerlofNemen_8', 'WerktijdenBepalen_9', 'EmotioneelMoeilijkeWerksituaties_11', 'EmotioneelVeeleisendWerk_12', 'EmotioneelBetrokkenBijWerk_13', 'PsychischeVermoeidheidDoorHetWerk_38', 'EmotioneelUitgeput_39', 'LeegVoelen_40', 'SOchtendsMoeVoelen_41', 'VergtVeelOmMetMensenTeWerken_42', 'UitgeputDoorWerk_43']
#Werkplekbepalen_10 wordt pas sinds 2022 gevraagd, dus daarom niet meegenomen

merged_df = merged_df.dropna(subset=columns_to_compare)

merged_df[columns_to_compare].info()

In [ ]:
correlation_matrix = merged_df[columns_to_compare].corr()
print(correlation_matrix)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix total data')
plt.show()

In [ ]:
# filter op alleen de totalen voor NL (alle branches)

merged_df_total = merged_df[merged_df['BedrijfskenmerkenSBI2008'] == 1081]

merged_df_total

In [ ]:
correlation_matrix_total = merged_df_total[columns_to_compare].corr()
print(correlation_matrix)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix_total, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix all economic activities')
plt.show()